# Modelando precios de casas - Navent Dataset

## Introducción

Vamos a trabajar en equipo para obtener el mejor resultado posible en la competencia de Valuación de casas de Fundacion Sadosky: https://metadata.fundacionsadosky.org.ar/competition/4/.

El objetivo será generar un modelo de regresión que nos acerque lo más posible al top ten de la competencia con la métrica seleccionada (RMSLE).

Antes de empezar vamos a tener que tener en cuenta algunos puntos:

- La competencia ya finalizó, por lo que no podemos cargar nuestros datos en la página y comparar directamente con los resultados del dataset de test. Por ello, dividiremos el dataset de training en dos: el que usaremos para modelar, y uno hold-out para validar que usaremos para comparar los mejores modelos que ustedes hayan generado.
- Vamos a trabajar en equipo, con la idea de que esto ayude a la comprensión del problema y a la búsquedas de estrategias para mejorar nuestro modelo, es sumamente importante que todos participen y discutan las decisiones que toman.
- Vamos a trabajar con este dataset en las próximas 3 clases, incorporando nuevas técnicas y conceptos para intentar mejorar nuestros modelos, por lo que no tienen que apurarse para resolver todo en la primera semana, tienen tiempo de analizar el problema e ir iterando en las posibles soluciones.
- Por último, recuerden que es un proceso iterativo, es recomendable que empiecen simple, lleguen a entrenar un modelo no tan complejo, obtengan un resultado, y luego vayan avanzando sobre eso.

## Actividades

1. Repasar el notebook introductorio provisto por Navent:
https://metadata.fundacionsadosky.org.ar/media/navent/metaDataNavent.ipynb

2. Investigar acerca de la métrica seleccionada (RMSLE): https://hrngok.github.io/posts/metrics/

3. Dividir el dataset en dos partes: una parte para training-validation, y una parte para hold-out, ésta última debe tener el 25% de los datos. Deben usar train_test_split de Scikit Learn con random_state=42 (esto es sólo para que todos tengamos el mismo split y las comparaciones entre sus modelos sean equivalentes)

4. Aplicar pre procesamiento que consideren necesario sobre los datos. Pueden usar como ejemplo lo que está hecho en el notebook del punto 1, pero sería interesante que tomen decisiones basados en un análisis propio: ¿agregarían o quitarían features? ¿Cambiarían algo del procesamiento de las features categóricas?

5. Modelar el problema. Deben entrenar modelos usando cross validation y grid/random search, para ello deben seleccionar que algoritmos de regresión van a probar: hasta ahora sólo hemos visto Regresión Lineal y Árboles de decisión, pero pueden seleccionar otro modelo lineal de la librería Scikit Learn: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model. Acá sería bueno que investiguen respecto a algoritmos que vayan a elegir: ¿Que aportan sobre una regresión lineal tradicional? ¿Qué hiperparámetros se pueden configurar? 

6. Luego de entrenar con validación cruzada y búsqueda de hiperparámetros, comprobar el funcionamiento del modelo sobre el dataset hold out. Recuerden que esto debe hacerse solo al final para comparar modelos, no para optimizar hiperparámetros, por lo que es el paso final del proceso.

7. Generar un notebook con explicaciones claras del proceso realizado, las decisiones que se tomaron, y el resultado obtenido. 


In [2]:
import pandas as pd
df = pd.read_csv('train.csv')
df.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0


In [11]:
nulls = pd.DataFrame(df.isnull().sum().sort_values(), columns=['nulls'])
nulls['porcentaje'] = round(100*nulls['nulls'] / len(df), 2)
nulls

,nulls,porcentaje
id,0,0.00
escuelascercanas,0,0.00
piscina,0,0.00
usosmultiples,0,0.00
gimnasio,0,0.00
fecha,0,0.00
centroscomercialescercanos,0,0.00
precio,0,0.00
tipodepropiedad,46,0.02
provincia,155,0.06


In [12]:
df_test = pd.read_csv('test.csv')

In [13]:
nulls = pd.DataFrame(df_test.isnull().sum().sort_values(), columns=['nulls'])
nulls['porcentaje'] = round(100*nulls['nulls'] / len(df), 2)
nulls

,nulls,porcentaje
id,0,0.00
piscina,0,0.00
usosmultiples,0,0.00
gimnasio,0,0.00
fecha,0,0.00
escuelascercanas,0,0.00
centroscomercialescercanos,0,0.00
tipodepropiedad,7,0.00
provincia,42,0.02
ciudad,83,0.03


In [9]:
X = df.drop(['precio'], axis=1)
y = df.precio

from sklearn.model_selection import train_test_split

X_1, X_hold_out, y_1, y_hold_out = train_test_split(X, y, test_size=0.25, random_state=42)


(60000, 22)
(135000, 22)
(45000, 22)


In [14]:
nulls = pd.DataFrame(X_hold_out.isnull().sum().sort_values(), columns=['nulls'])
nulls['porcentaje'] = round(100*nulls['nulls'] / len(df), 2)
nulls

,nulls,porcentaje
id,0,0.00
piscina,0,0.00
usosmultiples,0,0.00
gimnasio,0,0.00
fecha,0,0.00
escuelascercanas,0,0.00
centroscomercialescercanos,0,0.00
tipodepropiedad,14,0.01
provincia,43,0.02
ciudad,104,0.04


In [15]:
nulls = pd.DataFrame(X_1.isnull().sum().sort_values(), columns=['nulls'])
nulls['porcentaje'] = round(100*nulls['nulls'] / len(df), 2)
nulls

,nulls,porcentaje
id,0,0.00
piscina,0,0.00
usosmultiples,0,0.00
gimnasio,0,0.00
fecha,0,0.00
escuelascercanas,0,0.00
centroscomercialescercanos,0,0.00
tipodepropiedad,32,0.01
provincia,112,0.05
ciudad,268,0.11


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size=0.25, random_state=42)

print(X_hold_out.shape)
print(X_train.shape)
print(X_test.shape)

(60000, 22)
(135000, 22)
(45000, 22)


In [17]:
nulls = pd.DataFrame(X_train.isnull().sum().sort_values(), columns=['nulls'])
nulls['porcentaje'] = round(100*nulls['nulls'] / len(df), 2)
nulls

,nulls,porcentaje
id,0,0.00
piscina,0,0.00
usosmultiples,0,0.00
gimnasio,0,0.00
fecha,0,0.00
escuelascercanas,0,0.00
centroscomercialescercanos,0,0.00
tipodepropiedad,23,0.01
provincia,85,0.04
ciudad,204,0.08
